In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/shakespear/input.txt
/kaggle/input/shakes/input.txt


In [2]:
file_path = '/kaggle/input/shakespear/input.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read().lower()


In [3]:
import string
punc=string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('','',punc))

In [4]:
clean_text=remove_punc(text)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer


2024-06-09 13:29:23.395026: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 13:29:23.395155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 13:29:23.529807: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts([clean_text])

In [7]:
length=len(tokenizer.word_index)+1


In [8]:
input1=[]
for i in clean_text.split('\n'):
    sequence=tokenizer.texts_to_sequences([i])[0]
    for i in range(1,len(sequence)):
        input1.append(sequence[:i+1])


In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
max_length_of_input_node=max(len(x) for x in input1)
max_length_of_input_node

16

In [11]:
padded_sequence=pad_sequences(input1,maxlen=max_length_of_input_node,padding='pre')

In [12]:
x=padded_sequence[:,:-1]
y=padded_sequence[:,-1]
x

array([[   0,    0,    0, ...,    0,    0,   88],
       [   0,    0,    0, ...,    0,    0,  137],
       [   0,    0,    0, ...,    0,  137,   35],
       ...,
       [   0,    0,    0, ...,    0,    0, 1054],
       [   0,    0,    0, ...,    0, 1054,   25],
       [   0,    0,    0, ..., 1054,   25,  128]], dtype=int32)

In [13]:
y

array([ 267,   35,  968, ...,   25,  128, 1943], dtype=int32)

In [14]:
input_row_length=x.shape[1]
print(input_row_length)
print(y.shape)

15
(169869,)


In [15]:
from tensorflow.keras.utils import to_categorical
y_category=to_categorical(y,num_classes=length)
y_category.shape

(169869, 12849)

In [16]:
print(x.shape[0]) 
print(y_category.shape[0])

169869
169869


In [17]:
output_category=y_category.shape[1]
y_category.shape[1]
y_category.dtype
output_category

12849

In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Layer,Dense,LayerNormalization,MultiHeadAttention,Embedding,Dropout,Add,GlobalAveragePooling1D


In [19]:
class PositionalEncoding(Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(max_len, d_model)

    def positional_encoding(self, max_len, d_model):
        angle_rads = self.get_angles(np.arange(max_len)[:, np.newaxis],
                                     np.arange(d_model)[np.newaxis, :],
                                     d_model)
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]
        return inputs + self.pos_encoding[:, :seq_len, :]

In [20]:
max_len = max_length_of_input_node-1
d_model = 128
num_heads = 8
total_unique_words=len(tokenizer.word_index)+1
inputs=Input(shape=(x.shape[1],))
embedding = Embedding(input_dim=total_unique_words, output_dim=d_model)(inputs)
pos_encoding = PositionalEncoding(max_len, d_model)(embedding)
multi_head=MultiHeadAttention(num_heads=num_heads,key_dim=d_model,dropout=0.1)(pos_encoding,pos_encoding)
element_addition1=Add()([pos_encoding,multi_head])
layer_normalization1=LayerNormalization(epsilon=1e-6)(element_addition1)

ffd1=Dense(64,activation='relu')(layer_normalization1)
ffd2=Dense(128,activation='relu')(ffd1)
dropout=Dropout(0.50)(ffd2)
element_addition2=Add()([dropout,layer_normalization1])
layer_normalization2=LayerNormalization(epsilon=1e-6)(element_addition2)

x1 =GlobalAveragePooling1D()(layer_normalization2)
outputs=Dense(y_category.shape[1],activation='softmax')(x1)

model = Model(inputs=inputs, outputs=outputs)

In [21]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 15, 128)   │  1,644,672 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encoding │ (None, 15, 128)   │          0 │ embedding[0][0]   │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 15, 128)   │    527,488 │ positional_encod… │
│ (MultiHeadAttentio… │                   │            │ positional_encod… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 15, 128)   │          0 │ positional_encod… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 15, 128)   │        256 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 15, 64)    │      8,256 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 15, 128)   │      8,320 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 15, 128)   │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 15, 128)   │          0 │ dropout_1[0][0],  │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 15, 128)   │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 12849)     │  1,657,521 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,846,769 (14.67 MB)

 Trainable params: 3,846,769 (14.67 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.fit(x,y_category,batch_size=400,epochs=200)

Epoch 1/200
  5/425 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.0049 - loss: 9.3698     

I0000 00:00:1717939831.242443      83 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1717939831.262250      83 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


425/425 ━━━━━━━━━━━━━━━━━━━━ 24s 33ms/step - accuracy: 0.0290 - loss: 7.2437
Epoch 2/200
  3/425 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.0551 - loss: 6.5388

W0000 00:00:1717939845.395495      85 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.0622 - loss: 6.3483
Epoch 3/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.0878 - loss: 6.0251
Epoch 4/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.0991 - loss: 5.8102
Epoch 5/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.1063 - loss: 5.6509
Epoch 6/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.1137 - loss: 5.5086
Epoch 7/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.1200 - loss: 5.3603
Epoch 8/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.1270 - loss: 5.2419
Epoch 9/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.1328 - loss: 5.1232
Epoch 10/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.1416 - loss: 5.0149
Epoch 11/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.1490 - loss: 4.9056
Epoch 12/200
425/425 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.1544 - loss: 4.8164
Epoch 13/200
425/425 ━━━━━━━━━━━━━━━━━━━━

In [26]:
import time
import numpy as np

text1 =input('enter the text after you want to predict')
loop=int(input('How many words you want to predict'))

for i in range(loop):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text1])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=15, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text1 = text1 + " " + word
      print(text1)
      time.sleep(1)

enter the text after you want to predict king
How many words you want to predict 20


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
king edward
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
king edward iv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
king edward iv and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
king edward iv and i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
king edward iv and i am
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
king edward iv and i am content
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
king edward iv and i am content to
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
king edward iv and i am content to be
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
king edward iv and i am content to be your
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
king edward iv and i am content to be your concubine
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
king edward iv and i am content to be your concubine with
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
king edward iv and i am content to be your concubine with the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
king edward iv and i am content to be your concubine with the windy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0